In [1]:
# importing necessary libraries and dependencies
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn import metrics

Using TensorFlow backend.
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home

In [2]:
train_data_dir = 'sounds/train/train_sound/'
test_data_dir = 'sounds/test/test_sound/'

val_data_dir = 'sounds/val/val_sound/'

# reading the labels
train = pd.read_csv('sounds/labels/train.csv')
test  = pd.read_csv('sounds/labels/test.csv')

val  = pd.read_csv('sounds/labels/train.csv')

In [3]:
import os
import shutil

#Opening all 3 model predictions
model1pred = pd.read_csv("predictions/model1_a_all3_predictions.csv")
model2pred = pd.read_csv("predictions/model2_a_all3_predictions.csv")
model3pred = pd.read_csv("predictions/model3_a_all3_predictions.csv")

#Making the ID column as the index
model1pred.set_index("ID", inplace = True) 
model2pred.set_index("ID", inplace = True) 
model3pred.set_index("ID", inplace = True) 

#Printing out the head of all 3
print('-----------Model1 Pred-------------')
print(model1pred.head())

print('-----------Model2 Pred-------------')
print(model2pred.head())

print('-----------Model3 Pred-------------')
print(model3pred.head())

-----------Model1 Pred-------------
          ClassPred
ID                 
1          drilling
3   air_conditioner
10         drilling
12         gun_shot
15         gun_shot
-----------Model2 Pred-------------
        ClassPred
ID               
1        gun_shot
3   engine_idling
10       gun_shot
12          siren
15     jackhammer
-----------Model3 Pred-------------
          ClassPred
ID                 
1        jackhammer
3          gun_shot
10       jackhammer
12       jackhammer
15  air_conditioner


In [4]:
#model1pred['1']['ClassPred']
#model1pred.loc[1]['ClassPred']
#if model1pred.loc[1]['ClassPred']==model1pred.loc[1]['ClassPred']:
#    print('True')

#print(train)
#train.loc[train.ID == 8725, 'Class'] = "hello"
#print(train)

In [5]:
#If all 3 predictions match, then, MOVE the image to model1,model2,model3 and train_sound directory.
# Also, we need to overwrite the groundtruth with the predictions. For other 2 approaches, NO need of this, since it will use the ground truth from the train,csv

allfiles_dir = os.path.join(os.getcwd(),'sounds','train','train_sound_activelearning_30')    # Directory to move images from
target_dir1 = os.path.join(os.getcwd(),'sounds','train','model1')       # Directory to move images onto
target_dir2 = os.path.join(os.getcwd(),'sounds','train','model2')       # Directory to move images onto
target_dir3 = os.path.join(os.getcwd(),'sounds','train','model3')       # Directory to move images onto
target_dir4 = os.path.join(os.getcwd(),'sounds','train','train_sound')       # Directory to move images onto

for index, row in model1pred.iterrows():
    #print(model2pred.loc[index]['ClassPred'])
    if model1pred.loc[index]['ClassPred']==model2pred.loc[index]['ClassPred'] or model1pred.loc[index]['ClassPred']==model3pred.loc[index]['ClassPred'] or model2pred.loc[index]['ClassPred']==model3pred.loc[index]['ClassPred']:
        name=str(index)+'.wav'
        print(name +" : "+model1pred.loc[index]['ClassPred'])
        shutil.copy(os.path.join(allfiles_dir, name), os.path.join(target_dir1, name))
        shutil.copy(os.path.join(allfiles_dir, name), os.path.join(target_dir2, name))
        shutil.copy(os.path.join(allfiles_dir, name), os.path.join(target_dir3, name))
        shutil.move(os.path.join(allfiles_dir, name), os.path.join(target_dir4, name)) #Move finally
        
        #Changing the Class Prediction for this approach 'a'
        #train.loc[train.ID == index, 'Class'] = model1pred.loc[index]['ClassPred']

247.wav : gun_shot
391.wav : gun_shot
522.wav : gun_shot
619.wav : dog_bark
744.wav : gun_shot
816.wav : drilling
874.wav : drilling
904.wav : gun_shot
1059.wav : drilling
1188.wav : drilling
1270.wav : drilling
1709.wav : gun_shot
1787.wav : drilling
2020.wav : drilling
2446.wav : air_conditioner
3317.wav : air_conditioner
3529.wav : drilling
3595.wav : gun_shot
3696.wav : gun_shot
3788.wav : drilling
3870.wav : air_conditioner
4314.wav : gun_shot
4633.wav : drilling
4851.wav : drilling
4911.wav : gun_shot
4989.wav : gun_shot
5352.wav : gun_shot
5611.wav : drilling
5733.wav : drilling
5761.wav : dog_bark
6418.wav : gun_shot
6486.wav : drilling
6507.wav : gun_shot
7177.wav : drilling
8546.wav : drilling
8679.wav : air_conditioner
8713.wav : gun_shot


In [6]:
#Saving the train to a new CSV file since we changed the values
#train.to_csv('sounds/labels/temp_train.csv', sep='\t')

#Reading all the files as before, but loading the new train.csv file
train_data_dir = 'sounds/train/train_sound/'
test_data_dir = 'sounds/test/test_sound/'

val_data_dir = 'sounds/val/val_sound/'

# reading the labels
#train = pd.read_csv('sounds/labels/temp_train.csv')
train = pd.read_csv('sounds/labels/train.csv')
test  = pd.read_csv('sounds/labels/test.csv')

val  = pd.read_csv('sounds/labels/train.csv')

In [7]:
### Train: Function to delete values from CSV files for not found values

#Setting df as train (train.ierrows)
data_dir=train_data_dir

for index,row in train.iterrows():
    file_name = os.path.join(data_dir,str(row.ID)+'.wav')
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    except Exception as e:
        print(file_name)
        train=train.drop(index)

/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


sounds/train/train_sound/1.wav
sounds/train/train_sound/3.wav


/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


sounds/train/train_sound/10.wav
sounds/train/train_sound/12.wav
sounds/train/train_sound/15.wav


/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


...

/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


sounds/train/train_sound/8710.wav
sounds/train/train_sound/8714.wav


/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


sounds/train/train_sound/8720.wav
sounds/train/train_sound/8721.wav
sounds/train/train_sound/8722.wav
sounds/train/train_sound/8724.wav
sounds/train/train_sound/8725.wav
sounds/train/train_sound/8729.wav


/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [8]:
train.head()

,ID,Class
0,0,siren
2,2,drilling
4,4,dog_bark
5,6,children_playing
7,11,drilling


In [9]:
### Val: Function to delete values from CSV files for not found values

#Setting df as train (train.ierrows)
data_dir=val_data_dir

for index,row in val.iterrows():
    file_name = os.path.join(data_dir,str(row.ID)+'.wav')
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    except Exception as e:
        print(file_name)
        val=val.drop(index)

/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile fail

sounds/val/val_sound/0.wav
sounds/val/val_sound/1.wav
sounds/val/val_sound/2.wav
sounds/val/val_sound/3.wav
sounds/val/val_sound/4.wav


/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile fail

sounds/val/val_sound/6.wav
sounds/val/val_sound/10.wav
sounds/val/val_sound/11.wav
sounds/val/val_sound/12.wav
sounds/val/val_sound/15.wav


/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile fail

...

/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile fail

sounds/val/val_sound/8713.wav
sounds/val/val_sound/8714.wav
sounds/val/val_sound/8715.wav
sounds/val/val_sound/8716.wav
sounds/val/val_sound/8717.wav


/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


sounds/val/val_sound/8720.wav
sounds/val/val_sound/8723.wav
sounds/val/val_sound/8726.wav
sounds/val/val_sound/8727.wav
sounds/val/val_sound/8728.wav


/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/anandgokul18/anaconda3/envs/audio/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [10]:
val.head()

,ID,Class
15,24,air_conditioner
16,26,engine_idling
17,27,siren
19,33,jackhammer
32,49,drilling


In [11]:
# function to load files and extract features
def parser(row, data_dir):
    # setting path
    file_name = os.path.join(data_dir,str(row.ID)+'.wav')
    print(file_name)
    # check if the file is corrupted
    try:
        # here kaiser_fast is a technique used for faster extraction
        # X-> audio_time_series_data; sample_rate-> sampling rate
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        
        # extraccting Mel-Frequeny Cepstral Coeficients feature from data
        # y -> accepts time-series audio data; sr -> accepts sampling rate
        # n_mfccs -> no. of MFCCs to return
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis = 0)
    
    except Exception as e:
        print("Error encountered while parsing file: ", e)
        return None, None
    
    # store mfccs features
    feature = mfccs
    # store the respective id
    data_id = row.ID
    
    return [data_id, feature]

In [12]:
# function to load files and extract features
def parser_feature(row, data_dir):
    # setting path
    file_name = os.path.join(data_dir,str(row.ID)+'.wav')
    print(file_name)
    # check if the file is corrupted
    try:
        # here kaiser_fast is a technique used for faster extraction
        # X-> audio_time_series_data; sample_rate-> sampling rate
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        
        # extraccting Mel-Frequeny Cepstral Coeficients feature from data
        # y -> accepts time-series audio data; sr -> accepts sampling rate
        # n_mfccs -> no. of MFCCs to return
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis = 0)
    
    except Exception as e:
        print("Error encountered while parsing file: ", e)
        
        return None, None
    
    # store mfccs features
    feature = mfccs
    # store the respective id
    data_id = row.ID
    
    return feature

In [13]:
# function to load files and extract features
def parser_id(row, data_dir):
    # setting path
    file_name = os.path.join(data_dir,str(row.ID)+'.wav')
    print(file_name)
    # check if the file is corrupted
    try:
        # here kaiser_fast is a technique used for faster extraction
        # X-> audio_time_series_data; sample_rate-> sampling rate
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        
        # extraccting Mel-Frequeny Cepstral Coeficients feature from data
        # y -> accepts time-series audio data; sr -> accepts sampling rate
        # n_mfccs -> no. of MFCCs to return
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis = 0)
    
    except Exception as e:
        print("Error encountered while parsing file: ", e)
        return None, None
    
    # store mfccs features
    feature = mfccs
    # store the respective id
    data_id = row.ID
    
    return data_id

### Reading train.csv and storing into temp

In [14]:
# parsing train
temp = train.apply(parser,axis=1,data_dir=train_data_dir)
temp.columns = ['ID','feature']
temp['ID'] = train.apply(parser_id,axis=1,data_dir=train_data_dir)
temp['feature'] = train.apply(parser_feature,axis=1,data_dir=train_data_dir)

sounds/train/train_sound/0.wav
sounds/train/train_sound/0.wav
sounds/train/train_sound/2.wav
sounds/train/train_sound/4.wav
sounds/train/train_sound/6.wav
sounds/train/train_sound/11.wav
sounds/train/train_sound/17.wav
sounds/train/train_sound/18.wav
sounds/train/train_sound/19.wav
sounds/train/train_sound/22.wav
sounds/train/train_sound/35.wav
sounds/train/train_sound/36.wav
sounds/train/train_sound/37.wav
sounds/train/train_sound/40.wav
sounds/train/train_sound/42.wav
sounds/train/train_sound/43.wav
sounds/train/train_sound/44.wav
sounds/train/train_sound/46.wav
sounds/train/train_sound/47.wav
sounds/train/train_sound/52.wav
sounds/train/train_sound/56.wav
sounds/train/train_sound/65.wav
sounds/train/train_sound/67.wav
sounds/train/train_sound/68.wav
sounds/train/train_sound/74.wav
sounds/train/train_sound/76.wav
sounds/train/train_sound/77.wav
sounds/train/train_sound/79.wav
sounds/train/train_sound/80.wav
sounds/train/train_sound/87.wav
sounds/train/train_sound/90.wav
sounds/train/

sounds/train/train_sound/622.wav
sounds/train/train_sound/624.wav
sounds/train/train_sound/630.wav
sounds/train/train_sound/633.wav
sounds/train/train_sound/639.wav
sounds/train/train_sound/642.wav
sounds/train/train_sound/643.wav
sounds/train/train_sound/645.wav
sounds/train/train_sound/646.wav
sounds/train/train_sound/648.wav
sounds/train/train_sound/655.wav
sounds/train/train_sound/656.wav
sounds/train/train_sound/657.wav
sounds/train/train_sound/658.wav
sounds/train/train_sound/661.wav
sounds/train/train_sound/662.wav
sounds/train/train_sound/664.wav
sounds/train/train_sound/666.wav
sounds/train/train_sound/669.wav
sounds/train/train_sound/670.wav
sounds/train/train_sound/677.wav
sounds/train/train_sound/679.wav
sounds/train/train_sound/684.wav
sounds/train/train_sound/687.wav
sounds/train/train_sound/688.wav
sounds/train/train_sound/689.wav
sounds/train/train_sound/691.wav
sounds/train/train_sound/692.wav
sounds/train/train_sound/693.wav
sounds/train/train_sound/695.wav
sounds/tra

sounds/train/train_sound/1337.wav
sounds/train/train_sound/1339.wav
sounds/train/train_sound/1341.wav
sounds/train/train_sound/1345.wav
sounds/train/train_sound/1349.wav
sounds/train/train_sound/1350.wav
sounds/train/train_sound/1351.wav
sounds/train/train_sound/1352.wav
sounds/train/train_sound/1355.wav
sounds/train/train_sound/1357.wav
sounds/train/train_sound/1358.wav
sounds/train/train_sound/1361.wav
sounds/train/train_sound/1367.wav
sounds/train/train_sound/1371.wav
sounds/train/train_sound/1377.wav
sounds/train/train_sound/1378.wav
sounds/train/train_sound/1384.wav
sounds/train/train_sound/1385.wav
sounds/train/train_sound/1386.wav
sounds/train/train_sound/1393.wav
sounds/train/train_sound/1394.wav
sounds/train/train_sound/1396.wav
sounds/train/train_sound/1400.wav
sounds/train/train_sound/1405.wav
sounds/train/train_sound/1409.wav
sounds/train/train_sound/1412.wav
sounds/train/train_sound/1414.wav
sounds/train/train_sound/1415.wav
sounds/train/train_sound/1417.wav
sounds/train/t

sounds/train/train_sound/2025.wav
sounds/train/train_sound/2028.wav
sounds/train/train_sound/2029.wav
sounds/train/train_sound/2033.wav
sounds/train/train_sound/2048.wav
sounds/train/train_sound/2049.wav
sounds/train/train_sound/2050.wav
sounds/train/train_sound/2054.wav
sounds/train/train_sound/2056.wav
sounds/train/train_sound/2063.wav
sounds/train/train_sound/2064.wav
sounds/train/train_sound/2067.wav
sounds/train/train_sound/2078.wav
sounds/train/train_sound/2080.wav
sounds/train/train_sound/2081.wav
sounds/train/train_sound/2082.wav
sounds/train/train_sound/2083.wav
sounds/train/train_sound/2087.wav
sounds/train/train_sound/2090.wav
sounds/train/train_sound/2091.wav
sounds/train/train_sound/2095.wav
sounds/train/train_sound/2096.wav
sounds/train/train_sound/2101.wav
sounds/train/train_sound/2102.wav
sounds/train/train_sound/2103.wav
sounds/train/train_sound/2106.wav
sounds/train/train_sound/2113.wav
sounds/train/train_sound/2116.wav
sounds/train/train_sound/2121.wav
sounds/train/t

sounds/train/train_sound/2710.wav
sounds/train/train_sound/2711.wav
sounds/train/train_sound/2717.wav
sounds/train/train_sound/2718.wav
sounds/train/train_sound/2722.wav
sounds/train/train_sound/2725.wav
sounds/train/train_sound/2728.wav
sounds/train/train_sound/2730.wav
sounds/train/train_sound/2733.wav
sounds/train/train_sound/2736.wav
sounds/train/train_sound/2738.wav
sounds/train/train_sound/2740.wav
sounds/train/train_sound/2741.wav
sounds/train/train_sound/2744.wav
sounds/train/train_sound/2746.wav
sounds/train/train_sound/2747.wav
sounds/train/train_sound/2750.wav
sounds/train/train_sound/2752.wav
sounds/train/train_sound/2756.wav
sounds/train/train_sound/2760.wav
sounds/train/train_sound/2764.wav
sounds/train/train_sound/2769.wav
sounds/train/train_sound/2771.wav
sounds/train/train_sound/2773.wav
sounds/train/train_sound/2774.wav
sounds/train/train_sound/2785.wav
sounds/train/train_sound/2789.wav
sounds/train/train_sound/2792.wav
sounds/train/train_sound/2797.wav
sounds/train/t

...

sounds/train/train_sound/5848.wav
sounds/train/train_sound/5850.wav
sounds/train/train_sound/5854.wav
sounds/train/train_sound/5858.wav
sounds/train/train_sound/5859.wav
sounds/train/train_sound/5871.wav
sounds/train/train_sound/5876.wav
sounds/train/train_sound/5886.wav
sounds/train/train_sound/5890.wav
sounds/train/train_sound/5893.wav
sounds/train/train_sound/5897.wav
sounds/train/train_sound/5900.wav
sounds/train/train_sound/5901.wav
sounds/train/train_sound/5902.wav
sounds/train/train_sound/5906.wav
sounds/train/train_sound/5909.wav
sounds/train/train_sound/5912.wav
sounds/train/train_sound/5914.wav
sounds/train/train_sound/5921.wav
sounds/train/train_sound/5925.wav
sounds/train/train_sound/5927.wav
sounds/train/train_sound/5928.wav
sounds/train/train_sound/5933.wav
sounds/train/train_sound/5934.wav
sounds/train/train_sound/5936.wav
sounds/train/train_sound/5937.wav
sounds/train/train_sound/5940.wav
sounds/train/train_sound/5941.wav
sounds/train/train_sound/5943.wav
sounds/train/t

sounds/train/train_sound/6515.wav
sounds/train/train_sound/6521.wav
sounds/train/train_sound/6522.wav
sounds/train/train_sound/6524.wav
sounds/train/train_sound/6528.wav
sounds/train/train_sound/6529.wav
sounds/train/train_sound/6530.wav
sounds/train/train_sound/6537.wav
sounds/train/train_sound/6544.wav
sounds/train/train_sound/6547.wav
sounds/train/train_sound/6553.wav
sounds/train/train_sound/6555.wav
sounds/train/train_sound/6556.wav
sounds/train/train_sound/6557.wav
sounds/train/train_sound/6558.wav
sounds/train/train_sound/6559.wav
sounds/train/train_sound/6562.wav
sounds/train/train_sound/6566.wav
sounds/train/train_sound/6568.wav
sounds/train/train_sound/6570.wav
sounds/train/train_sound/6574.wav
sounds/train/train_sound/6575.wav
sounds/train/train_sound/6579.wav
sounds/train/train_sound/6580.wav
sounds/train/train_sound/6583.wav
sounds/train/train_sound/6589.wav
sounds/train/train_sound/6595.wav
sounds/train/train_sound/6596.wav
sounds/train/train_sound/6599.wav
sounds/train/t

sounds/train/train_sound/7142.wav
sounds/train/train_sound/7143.wav
sounds/train/train_sound/7144.wav
sounds/train/train_sound/7148.wav
sounds/train/train_sound/7150.wav
sounds/train/train_sound/7153.wav
sounds/train/train_sound/7155.wav
sounds/train/train_sound/7156.wav
sounds/train/train_sound/7157.wav
sounds/train/train_sound/7161.wav
sounds/train/train_sound/7162.wav
sounds/train/train_sound/7171.wav
sounds/train/train_sound/7172.wav
sounds/train/train_sound/7174.wav
sounds/train/train_sound/7176.wav
sounds/train/train_sound/7177.wav
sounds/train/train_sound/7179.wav
sounds/train/train_sound/7180.wav
sounds/train/train_sound/7185.wav
sounds/train/train_sound/7191.wav
sounds/train/train_sound/7194.wav
sounds/train/train_sound/7195.wav
sounds/train/train_sound/7196.wav
sounds/train/train_sound/7197.wav
sounds/train/train_sound/7198.wav
sounds/train/train_sound/7202.wav
sounds/train/train_sound/7203.wav
sounds/train/train_sound/7204.wav
sounds/train/train_sound/7205.wav
sounds/train/t

sounds/train/train_sound/7830.wav
sounds/train/train_sound/7831.wav
sounds/train/train_sound/7833.wav
sounds/train/train_sound/7835.wav
sounds/train/train_sound/7838.wav
sounds/train/train_sound/7839.wav
sounds/train/train_sound/7842.wav
sounds/train/train_sound/7850.wav
sounds/train/train_sound/7855.wav
sounds/train/train_sound/7856.wav
sounds/train/train_sound/7862.wav
sounds/train/train_sound/7867.wav
sounds/train/train_sound/7869.wav
sounds/train/train_sound/7871.wav
sounds/train/train_sound/7872.wav
sounds/train/train_sound/7873.wav
sounds/train/train_sound/7875.wav
sounds/train/train_sound/7876.wav
sounds/train/train_sound/7879.wav
sounds/train/train_sound/7881.wav
sounds/train/train_sound/7883.wav
sounds/train/train_sound/7890.wav
sounds/train/train_sound/7891.wav
sounds/train/train_sound/7892.wav
sounds/train/train_sound/7897.wav
sounds/train/train_sound/7900.wav
sounds/train/train_sound/7904.wav
sounds/train/train_sound/7909.wav
sounds/train/train_sound/7911.wav
sounds/train/t

sounds/train/train_sound/8510.wav
sounds/train/train_sound/8511.wav
sounds/train/train_sound/8514.wav
sounds/train/train_sound/8515.wav
sounds/train/train_sound/8522.wav
sounds/train/train_sound/8523.wav
sounds/train/train_sound/8524.wav
sounds/train/train_sound/8526.wav
sounds/train/train_sound/8531.wav
sounds/train/train_sound/8534.wav
sounds/train/train_sound/8535.wav
sounds/train/train_sound/8536.wav
sounds/train/train_sound/8538.wav
sounds/train/train_sound/8539.wav
sounds/train/train_sound/8540.wav
sounds/train/train_sound/8542.wav
sounds/train/train_sound/8543.wav
sounds/train/train_sound/8544.wav
sounds/train/train_sound/8546.wav
sounds/train/train_sound/8553.wav
sounds/train/train_sound/8558.wav
sounds/train/train_sound/8560.wav
sounds/train/train_sound/8562.wav
sounds/train/train_sound/8564.wav
sounds/train/train_sound/8565.wav
sounds/train/train_sound/8569.wav
sounds/train/train_sound/8579.wav
sounds/train/train_sound/8580.wav
sounds/train/train_sound/8581.wav
sounds/train/t

In [15]:
# adding Class to 'temp'
temp['Class'] = train['Class']

In [16]:
type(temp)

pandas.core.series.Series

### Reading Validation train.csv and storing into temp_val

In [17]:
# parsing train
temp_val = val.apply(parser,axis=1,data_dir=val_data_dir)
temp_val.columns = ['ID','feature']
temp_val['ID'] = val.apply(parser_id,axis=1,data_dir=val_data_dir)
temp_val['feature'] = val.apply(parser_feature,axis=1,data_dir=val_data_dir)

sounds/val/val_sound/24.wav
sounds/val/val_sound/24.wav
sounds/val/val_sound/26.wav
sounds/val/val_sound/27.wav
sounds/val/val_sound/33.wav
sounds/val/val_sound/49.wav
sounds/val/val_sound/50.wav
sounds/val/val_sound/54.wav
sounds/val/val_sound/59.wav
sounds/val/val_sound/60.wav
sounds/val/val_sound/61.wav
sounds/val/val_sound/63.wav
sounds/val/val_sound/66.wav
sounds/val/val_sound/72.wav
sounds/val/val_sound/78.wav
sounds/val/val_sound/81.wav
sounds/val/val_sound/82.wav
sounds/val/val_sound/83.wav
sounds/val/val_sound/84.wav
sounds/val/val_sound/92.wav
sounds/val/val_sound/105.wav
sounds/val/val_sound/123.wav
sounds/val/val_sound/132.wav
sounds/val/val_sound/134.wav
sounds/val/val_sound/140.wav
sounds/val/val_sound/149.wav
sounds/val/val_sound/153.wav
sounds/val/val_sound/160.wav
sounds/val/val_sound/175.wav
sounds/val/val_sound/208.wav
sounds/val/val_sound/211.wav
sounds/val/val_sound/218.wav
sounds/val/val_sound/224.wav
sounds/val/val_sound/232.wav
sounds/val/val_sound/250.wav
sound

sounds/val/val_sound/2038.wav
sounds/val/val_sound/2051.wav
sounds/val/val_sound/2055.wav
sounds/val/val_sound/2057.wav
sounds/val/val_sound/2066.wav
sounds/val/val_sound/2073.wav
sounds/val/val_sound/2077.wav
sounds/val/val_sound/2099.wav
sounds/val/val_sound/2105.wav
sounds/val/val_sound/2108.wav
sounds/val/val_sound/2129.wav
sounds/val/val_sound/2138.wav
sounds/val/val_sound/2139.wav
sounds/val/val_sound/2150.wav
sounds/val/val_sound/2152.wav
sounds/val/val_sound/2169.wav
sounds/val/val_sound/2174.wav
sounds/val/val_sound/2191.wav
sounds/val/val_sound/2201.wav
sounds/val/val_sound/2210.wav
sounds/val/val_sound/2216.wav
sounds/val/val_sound/2219.wav
sounds/val/val_sound/2224.wav
sounds/val/val_sound/2233.wav
sounds/val/val_sound/2241.wav
sounds/val/val_sound/2251.wav
sounds/val/val_sound/2275.wav
sounds/val/val_sound/2277.wav
sounds/val/val_sound/2287.wav
sounds/val/val_sound/2290.wav
sounds/val/val_sound/2293.wav
sounds/val/val_sound/2310.wav
sounds/val/val_sound/2330.wav
sounds/val

sounds/val/val_sound/4335.wav
sounds/val/val_sound/4338.wav
sounds/val/val_sound/4352.wav
sounds/val/val_sound/4356.wav
sounds/val/val_sound/4362.wav
sounds/val/val_sound/4363.wav
sounds/val/val_sound/4369.wav
sounds/val/val_sound/4396.wav
sounds/val/val_sound/4401.wav
sounds/val/val_sound/4415.wav
sounds/val/val_sound/4418.wav
sounds/val/val_sound/4430.wav
sounds/val/val_sound/4442.wav
sounds/val/val_sound/4451.wav
sounds/val/val_sound/4454.wav
sounds/val/val_sound/4459.wav
sounds/val/val_sound/4462.wav
sounds/val/val_sound/4472.wav
sounds/val/val_sound/4473.wav
sounds/val/val_sound/4474.wav
sounds/val/val_sound/4475.wav
sounds/val/val_sound/4486.wav
sounds/val/val_sound/4491.wav
sounds/val/val_sound/4493.wav
sounds/val/val_sound/4527.wav
sounds/val/val_sound/4535.wav
sounds/val/val_sound/4536.wav
sounds/val/val_sound/4539.wav
sounds/val/val_sound/4540.wav
sounds/val/val_sound/4561.wav
sounds/val/val_sound/4566.wav
sounds/val/val_sound/4573.wav
sounds/val/val_sound/4574.wav
sounds/val

sounds/val/val_sound/6674.wav
sounds/val/val_sound/6679.wav
sounds/val/val_sound/6694.wav
sounds/val/val_sound/6712.wav
sounds/val/val_sound/6723.wav
sounds/val/val_sound/6725.wav
sounds/val/val_sound/6731.wav
sounds/val/val_sound/6739.wav
sounds/val/val_sound/6749.wav
sounds/val/val_sound/6750.wav
sounds/val/val_sound/6754.wav
sounds/val/val_sound/6765.wav
sounds/val/val_sound/6779.wav
sounds/val/val_sound/6802.wav
sounds/val/val_sound/6815.wav
sounds/val/val_sound/6829.wav
sounds/val/val_sound/6832.wav
sounds/val/val_sound/6841.wav
sounds/val/val_sound/6847.wav
sounds/val/val_sound/6865.wav
sounds/val/val_sound/6869.wav
sounds/val/val_sound/6875.wav
sounds/val/val_sound/6881.wav
sounds/val/val_sound/6882.wav
sounds/val/val_sound/6886.wav
sounds/val/val_sound/6888.wav
sounds/val/val_sound/6901.wav
sounds/val/val_sound/6902.wav
sounds/val/val_sound/6929.wav
sounds/val/val_sound/6931.wav
sounds/val/val_sound/6939.wav
sounds/val/val_sound/6947.wav
sounds/val/val_sound/6948.wav
sounds/val

sounds/val/val_sound/149.wav
sounds/val/val_sound/153.wav
sounds/val/val_sound/160.wav
sounds/val/val_sound/175.wav
sounds/val/val_sound/208.wav
sounds/val/val_sound/211.wav
sounds/val/val_sound/218.wav
sounds/val/val_sound/224.wav
sounds/val/val_sound/232.wav
sounds/val/val_sound/250.wav
sounds/val/val_sound/271.wav
sounds/val/val_sound/272.wav
sounds/val/val_sound/280.wav
sounds/val/val_sound/285.wav
sounds/val/val_sound/286.wav
sounds/val/val_sound/299.wav
sounds/val/val_sound/306.wav
sounds/val/val_sound/317.wav
sounds/val/val_sound/322.wav
sounds/val/val_sound/326.wav
sounds/val/val_sound/333.wav
sounds/val/val_sound/339.wav
sounds/val/val_sound/342.wav
sounds/val/val_sound/355.wav
sounds/val/val_sound/367.wav
sounds/val/val_sound/368.wav
sounds/val/val_sound/417.wav
sounds/val/val_sound/427.wav
sounds/val/val_sound/435.wav
sounds/val/val_sound/436.wav
sounds/val/val_sound/452.wav
sounds/val/val_sound/454.wav
sounds/val/val_sound/471.wav
sounds/val/val_sound/482.wav
sounds/val/val

...

sounds/val/val_sound/6888.wav
sounds/val/val_sound/6901.wav
sounds/val/val_sound/6902.wav
sounds/val/val_sound/6929.wav
sounds/val/val_sound/6931.wav
sounds/val/val_sound/6939.wav
sounds/val/val_sound/6947.wav
sounds/val/val_sound/6948.wav
sounds/val/val_sound/6955.wav
sounds/val/val_sound/6956.wav
sounds/val/val_sound/6975.wav
sounds/val/val_sound/6981.wav
sounds/val/val_sound/6984.wav
sounds/val/val_sound/6999.wav
sounds/val/val_sound/7000.wav
sounds/val/val_sound/7008.wav
sounds/val/val_sound/7025.wav
sounds/val/val_sound/7032.wav
sounds/val/val_sound/7039.wav
sounds/val/val_sound/7048.wav
sounds/val/val_sound/7063.wav
sounds/val/val_sound/7065.wav
sounds/val/val_sound/7066.wav
sounds/val/val_sound/7073.wav
sounds/val/val_sound/7077.wav
sounds/val/val_sound/7079.wav
sounds/val/val_sound/7091.wav
sounds/val/val_sound/7124.wav
sounds/val/val_sound/7140.wav
sounds/val/val_sound/7145.wav
sounds/val/val_sound/7151.wav
sounds/val/val_sound/7154.wav
sounds/val/val_sound/7169.wav
sounds/val

sounds/val/val_sound/355.wav
sounds/val/val_sound/367.wav
sounds/val/val_sound/368.wav
sounds/val/val_sound/417.wav
sounds/val/val_sound/427.wav
sounds/val/val_sound/435.wav
sounds/val/val_sound/436.wav
sounds/val/val_sound/452.wav
sounds/val/val_sound/454.wav
sounds/val/val_sound/471.wav
sounds/val/val_sound/482.wav
sounds/val/val_sound/488.wav
sounds/val/val_sound/500.wav
sounds/val/val_sound/501.wav
sounds/val/val_sound/506.wav
sounds/val/val_sound/509.wav
sounds/val/val_sound/512.wav
sounds/val/val_sound/520.wav
sounds/val/val_sound/525.wav
sounds/val/val_sound/542.wav
sounds/val/val_sound/551.wav
sounds/val/val_sound/554.wav
sounds/val/val_sound/559.wav
sounds/val/val_sound/560.wav
sounds/val/val_sound/565.wav
sounds/val/val_sound/566.wav
sounds/val/val_sound/568.wav
sounds/val/val_sound/579.wav
sounds/val/val_sound/580.wav
sounds/val/val_sound/585.wav
sounds/val/val_sound/605.wav
sounds/val/val_sound/621.wav
sounds/val/val_sound/623.wav
sounds/val/val_sound/651.wav
sounds/val/val

sounds/val/val_sound/2445.wav
sounds/val/val_sound/2454.wav
sounds/val/val_sound/2455.wav
sounds/val/val_sound/2463.wav
sounds/val/val_sound/2494.wav
sounds/val/val_sound/2499.wav
sounds/val/val_sound/2511.wav
sounds/val/val_sound/2523.wav
sounds/val/val_sound/2524.wav
sounds/val/val_sound/2541.wav
sounds/val/val_sound/2549.wav
sounds/val/val_sound/2550.wav
sounds/val/val_sound/2560.wav
sounds/val/val_sound/2569.wav
sounds/val/val_sound/2571.wav
sounds/val/val_sound/2576.wav
sounds/val/val_sound/2588.wav
sounds/val/val_sound/2595.wav
sounds/val/val_sound/2598.wav
sounds/val/val_sound/2608.wav
sounds/val/val_sound/2613.wav
sounds/val/val_sound/2616.wav
sounds/val/val_sound/2620.wav
sounds/val/val_sound/2636.wav
sounds/val/val_sound/2647.wav
sounds/val/val_sound/2651.wav
sounds/val/val_sound/2659.wav
sounds/val/val_sound/2665.wav
sounds/val/val_sound/2674.wav
sounds/val/val_sound/2677.wav
sounds/val/val_sound/2679.wav
sounds/val/val_sound/2721.wav
sounds/val/val_sound/2723.wav
sounds/val

sounds/val/val_sound/4698.wav
sounds/val/val_sound/4703.wav
sounds/val/val_sound/4722.wav
sounds/val/val_sound/4743.wav
sounds/val/val_sound/4746.wav
sounds/val/val_sound/4750.wav
sounds/val/val_sound/4754.wav
sounds/val/val_sound/4755.wav
sounds/val/val_sound/4758.wav
sounds/val/val_sound/4767.wav
sounds/val/val_sound/4783.wav
sounds/val/val_sound/4785.wav
sounds/val/val_sound/4788.wav
sounds/val/val_sound/4823.wav
sounds/val/val_sound/4829.wav
sounds/val/val_sound/4835.wav
sounds/val/val_sound/4840.wav
sounds/val/val_sound/4846.wav
sounds/val/val_sound/4857.wav
sounds/val/val_sound/4861.wav
sounds/val/val_sound/4870.wav
sounds/val/val_sound/4873.wav
sounds/val/val_sound/4881.wav
sounds/val/val_sound/4890.wav
sounds/val/val_sound/4900.wav
sounds/val/val_sound/4906.wav
sounds/val/val_sound/4909.wav
sounds/val/val_sound/4912.wav
sounds/val/val_sound/4920.wav
sounds/val/val_sound/4924.wav
sounds/val/val_sound/4927.wav
sounds/val/val_sound/4994.wav
sounds/val/val_sound/4997.wav
sounds/val

sounds/val/val_sound/7048.wav
sounds/val/val_sound/7063.wav
sounds/val/val_sound/7065.wav
sounds/val/val_sound/7066.wav
sounds/val/val_sound/7073.wav
sounds/val/val_sound/7077.wav
sounds/val/val_sound/7079.wav
sounds/val/val_sound/7091.wav
sounds/val/val_sound/7124.wav
sounds/val/val_sound/7140.wav
sounds/val/val_sound/7145.wav
sounds/val/val_sound/7151.wav
sounds/val/val_sound/7154.wav
sounds/val/val_sound/7169.wav
sounds/val/val_sound/7175.wav
sounds/val/val_sound/7187.wav
sounds/val/val_sound/7200.wav
sounds/val/val_sound/7211.wav
sounds/val/val_sound/7215.wav
sounds/val/val_sound/7218.wav
sounds/val/val_sound/7221.wav
sounds/val/val_sound/7226.wav
sounds/val/val_sound/7233.wav
sounds/val/val_sound/7235.wav
sounds/val/val_sound/7241.wav
sounds/val/val_sound/7249.wav
sounds/val/val_sound/7265.wav
sounds/val/val_sound/7280.wav
sounds/val/val_sound/7287.wav
sounds/val/val_sound/7297.wav
sounds/val/val_sound/7315.wav
sounds/val/val_sound/7328.wav
sounds/val/val_sound/7329.wav
sounds/val

In [18]:
# adding Class to 'temp'
temp_val['Class'] = val['Class']

In [19]:
type(temp_val)

pandas.core.series.Series

###  Reading test.csv and storing into temp_test

In [20]:
# parsing test
temp_test = test.apply(parser, axis=1,data_dir=test_data_dir)
temp_test.columns = ['ID', 'feature']

#temp_test['ID'] = test.apply(parser_id,axis=1,data_dir=test_data_dir)
#temp_test['feature'] = test.apply(parser,axis=1,data_dir=test_data_dir)

sounds/test/test_sound/5.wav
sounds/test/test_sound/7.wav
sounds/test/test_sound/8.wav
sounds/test/test_sound/9.wav
sounds/test/test_sound/13.wav
sounds/test/test_sound/14.wav
sounds/test/test_sound/16.wav
sounds/test/test_sound/21.wav
sounds/test/test_sound/23.wav
sounds/test/test_sound/25.wav
sounds/test/test_sound/28.wav
sounds/test/test_sound/29.wav
sounds/test/test_sound/30.wav
sounds/test/test_sound/31.wav
sounds/test/test_sound/34.wav
sounds/test/test_sound/39.wav
sounds/test/test_sound/41.wav
sounds/test/test_sound/51.wav
sounds/test/test_sound/53.wav
sounds/test/test_sound/55.wav
sounds/test/test_sound/58.wav
sounds/test/test_sound/62.wav
sounds/test/test_sound/64.wav
sounds/test/test_sound/69.wav
sounds/test/test_sound/70.wav
sounds/test/test_sound/73.wav
sounds/test/test_sound/75.wav
sounds/test/test_sound/85.wav
sounds/test/test_sound/86.wav
sounds/test/test_sound/89.wav
sounds/test/test_sound/94.wav
sounds/test/test_sound/95.wav
sounds/test/test_sound/96.wav
sounds/test/te

sounds/test/test_sound/795.wav
sounds/test/test_sound/797.wav
sounds/test/test_sound/800.wav
sounds/test/test_sound/805.wav
sounds/test/test_sound/806.wav
sounds/test/test_sound/808.wav
sounds/test/test_sound/814.wav
sounds/test/test_sound/815.wav
sounds/test/test_sound/818.wav
sounds/test/test_sound/822.wav
sounds/test/test_sound/823.wav
sounds/test/test_sound/827.wav
sounds/test/test_sound/828.wav
sounds/test/test_sound/831.wav
sounds/test/test_sound/836.wav
sounds/test/test_sound/837.wav
sounds/test/test_sound/842.wav
sounds/test/test_sound/846.wav
sounds/test/test_sound/847.wav
sounds/test/test_sound/852.wav
sounds/test/test_sound/856.wav
sounds/test/test_sound/861.wav
sounds/test/test_sound/862.wav
sounds/test/test_sound/866.wav
sounds/test/test_sound/873.wav
sounds/test/test_sound/877.wav
sounds/test/test_sound/879.wav
sounds/test/test_sound/880.wav
sounds/test/test_sound/888.wav
sounds/test/test_sound/890.wav
sounds/test/test_sound/895.wav
sounds/test/test_sound/897.wav
sounds/t

sounds/test/test_sound/1427.wav
sounds/test/test_sound/1429.wav
sounds/test/test_sound/1430.wav
sounds/test/test_sound/1433.wav
sounds/test/test_sound/1434.wav
sounds/test/test_sound/1438.wav
sounds/test/test_sound/1439.wav
sounds/test/test_sound/1440.wav
sounds/test/test_sound/1441.wav
sounds/test/test_sound/1442.wav
sounds/test/test_sound/1443.wav
sounds/test/test_sound/1445.wav
sounds/test/test_sound/1451.wav
sounds/test/test_sound/1453.wav
sounds/test/test_sound/1455.wav
sounds/test/test_sound/1456.wav
sounds/test/test_sound/1457.wav
sounds/test/test_sound/1463.wav
sounds/test/test_sound/1464.wav
sounds/test/test_sound/1465.wav
sounds/test/test_sound/1466.wav
sounds/test/test_sound/1474.wav
sounds/test/test_sound/1476.wav
sounds/test/test_sound/1482.wav
sounds/test/test_sound/1484.wav
sounds/test/test_sound/1486.wav
sounds/test/test_sound/1487.wav
sounds/test/test_sound/1490.wav
sounds/test/test_sound/1494.wav
sounds/test/test_sound/1495.wav
sounds/test/test_sound/1499.wav
sounds/t

sounds/test/test_sound/2109.wav
sounds/test/test_sound/2110.wav
sounds/test/test_sound/2111.wav
sounds/test/test_sound/2112.wav
sounds/test/test_sound/2114.wav
sounds/test/test_sound/2115.wav
sounds/test/test_sound/2118.wav
sounds/test/test_sound/2119.wav
sounds/test/test_sound/2120.wav
sounds/test/test_sound/2128.wav
sounds/test/test_sound/2131.wav
sounds/test/test_sound/2136.wav
sounds/test/test_sound/2137.wav
sounds/test/test_sound/2143.wav
sounds/test/test_sound/2146.wav
sounds/test/test_sound/2147.wav
sounds/test/test_sound/2151.wav
sounds/test/test_sound/2153.wav
sounds/test/test_sound/2155.wav
sounds/test/test_sound/2158.wav
sounds/test/test_sound/2159.wav
sounds/test/test_sound/2161.wav
sounds/test/test_sound/2164.wav
sounds/test/test_sound/2165.wav
sounds/test/test_sound/2166.wav
sounds/test/test_sound/2167.wav
sounds/test/test_sound/2172.wav
sounds/test/test_sound/2175.wav
sounds/test/test_sound/2176.wav
sounds/test/test_sound/2177.wav
sounds/test/test_sound/2178.wav
sounds/t

sounds/test/test_sound/2782.wav
sounds/test/test_sound/2786.wav
sounds/test/test_sound/2793.wav
sounds/test/test_sound/2796.wav
sounds/test/test_sound/2798.wav
sounds/test/test_sound/2799.wav
sounds/test/test_sound/2803.wav
sounds/test/test_sound/2804.wav
sounds/test/test_sound/2806.wav
sounds/test/test_sound/2809.wav
sounds/test/test_sound/2810.wav
sounds/test/test_sound/2812.wav
sounds/test/test_sound/2814.wav
sounds/test/test_sound/2816.wav
sounds/test/test_sound/2817.wav
sounds/test/test_sound/2818.wav
sounds/test/test_sound/2820.wav
sounds/test/test_sound/2822.wav
sounds/test/test_sound/2823.wav
sounds/test/test_sound/2826.wav
sounds/test/test_sound/2827.wav
sounds/test/test_sound/2833.wav
sounds/test/test_sound/2834.wav
sounds/test/test_sound/2838.wav
sounds/test/test_sound/2839.wav
sounds/test/test_sound/2841.wav
sounds/test/test_sound/2843.wav
sounds/test/test_sound/2844.wav
sounds/test/test_sound/2845.wav
sounds/test/test_sound/2847.wav
sounds/test/test_sound/2848.wav
sounds/t

...

sounds/test/test_sound/5468.wav
sounds/test/test_sound/5469.wav
sounds/test/test_sound/5470.wav
sounds/test/test_sound/5471.wav
sounds/test/test_sound/5474.wav
sounds/test/test_sound/5475.wav
sounds/test/test_sound/5478.wav
sounds/test/test_sound/5479.wav
sounds/test/test_sound/5482.wav
sounds/test/test_sound/5484.wav
sounds/test/test_sound/5485.wav
sounds/test/test_sound/5490.wav
sounds/test/test_sound/5491.wav
sounds/test/test_sound/5496.wav
sounds/test/test_sound/5500.wav
sounds/test/test_sound/5501.wav
sounds/test/test_sound/5502.wav
sounds/test/test_sound/5506.wav
sounds/test/test_sound/5510.wav
sounds/test/test_sound/5516.wav
sounds/test/test_sound/5517.wav
sounds/test/test_sound/5520.wav
sounds/test/test_sound/5525.wav
sounds/test/test_sound/5527.wav
sounds/test/test_sound/5529.wav
sounds/test/test_sound/5531.wav
sounds/test/test_sound/5532.wav
sounds/test/test_sound/5536.wav
sounds/test/test_sound/5538.wav
sounds/test/test_sound/5539.wav
sounds/test/test_sound/5541.wav
sounds/t

sounds/test/test_sound/6146.wav
sounds/test/test_sound/6147.wav
sounds/test/test_sound/6148.wav
sounds/test/test_sound/6150.wav
sounds/test/test_sound/6155.wav
sounds/test/test_sound/6157.wav
sounds/test/test_sound/6164.wav
sounds/test/test_sound/6166.wav
sounds/test/test_sound/6167.wav
sounds/test/test_sound/6168.wav
sounds/test/test_sound/6176.wav
sounds/test/test_sound/6178.wav
sounds/test/test_sound/6182.wav
sounds/test/test_sound/6184.wav
sounds/test/test_sound/6185.wav
sounds/test/test_sound/6186.wav
sounds/test/test_sound/6190.wav
sounds/test/test_sound/6192.wav
sounds/test/test_sound/6195.wav
sounds/test/test_sound/6199.wav
sounds/test/test_sound/6200.wav
sounds/test/test_sound/6201.wav
sounds/test/test_sound/6202.wav
sounds/test/test_sound/6203.wav
sounds/test/test_sound/6207.wav
sounds/test/test_sound/6208.wav
sounds/test/test_sound/6209.wav
sounds/test/test_sound/6216.wav
sounds/test/test_sound/6218.wav
sounds/test/test_sound/6221.wav
sounds/test/test_sound/6223.wav
sounds/t

sounds/test/test_sound/6843.wav
sounds/test/test_sound/6845.wav
sounds/test/test_sound/6848.wav
sounds/test/test_sound/6849.wav
sounds/test/test_sound/6854.wav
sounds/test/test_sound/6858.wav
sounds/test/test_sound/6859.wav
sounds/test/test_sound/6861.wav
sounds/test/test_sound/6862.wav
sounds/test/test_sound/6863.wav
sounds/test/test_sound/6867.wav
sounds/test/test_sound/6868.wav
sounds/test/test_sound/6870.wav
sounds/test/test_sound/6873.wav
sounds/test/test_sound/6874.wav
sounds/test/test_sound/6876.wav
sounds/test/test_sound/6884.wav
sounds/test/test_sound/6885.wav
sounds/test/test_sound/6887.wav
sounds/test/test_sound/6890.wav
sounds/test/test_sound/6894.wav
sounds/test/test_sound/6897.wav
sounds/test/test_sound/6899.wav
sounds/test/test_sound/6904.wav
sounds/test/test_sound/6905.wav
sounds/test/test_sound/6913.wav
sounds/test/test_sound/6916.wav
sounds/test/test_sound/6917.wav
sounds/test/test_sound/6919.wav
sounds/test/test_sound/6921.wav
sounds/test/test_sound/6924.wav
sounds/t

sounds/test/test_sound/7557.wav
sounds/test/test_sound/7559.wav
sounds/test/test_sound/7562.wav
sounds/test/test_sound/7564.wav
sounds/test/test_sound/7571.wav
sounds/test/test_sound/7572.wav
sounds/test/test_sound/7575.wav
sounds/test/test_sound/7583.wav
sounds/test/test_sound/7587.wav
sounds/test/test_sound/7588.wav
sounds/test/test_sound/7590.wav
sounds/test/test_sound/7591.wav
sounds/test/test_sound/7594.wav
sounds/test/test_sound/7603.wav
sounds/test/test_sound/7605.wav
sounds/test/test_sound/7606.wav
sounds/test/test_sound/7608.wav
sounds/test/test_sound/7615.wav
sounds/test/test_sound/7616.wav
sounds/test/test_sound/7621.wav
sounds/test/test_sound/7624.wav
sounds/test/test_sound/7627.wav
sounds/test/test_sound/7628.wav
sounds/test/test_sound/7630.wav
sounds/test/test_sound/7632.wav
sounds/test/test_sound/7635.wav
sounds/test/test_sound/7637.wav
sounds/test/test_sound/7640.wav
sounds/test/test_sound/7643.wav
sounds/test/test_sound/7647.wav
sounds/test/test_sound/7649.wav
sounds/t

sounds/test/test_sound/8245.wav
sounds/test/test_sound/8248.wav
sounds/test/test_sound/8250.wav
sounds/test/test_sound/8254.wav
sounds/test/test_sound/8255.wav
sounds/test/test_sound/8256.wav
sounds/test/test_sound/8267.wav
sounds/test/test_sound/8269.wav
sounds/test/test_sound/8273.wav
sounds/test/test_sound/8275.wav
sounds/test/test_sound/8276.wav
sounds/test/test_sound/8278.wav
sounds/test/test_sound/8281.wav
sounds/test/test_sound/8284.wav
sounds/test/test_sound/8286.wav
sounds/test/test_sound/8288.wav
sounds/test/test_sound/8290.wav
sounds/test/test_sound/8291.wav
sounds/test/test_sound/8292.wav
sounds/test/test_sound/8295.wav
sounds/test/test_sound/8298.wav
sounds/test/test_sound/8301.wav
sounds/test/test_sound/8303.wav
sounds/test/test_sound/8304.wav
sounds/test/test_sound/8309.wav
sounds/test/test_sound/8312.wav
sounds/test/test_sound/8316.wav
sounds/test/test_sound/8319.wav
sounds/test/test_sound/8320.wav
sounds/test/test_sound/8321.wav
sounds/test/test_sound/8326.wav
sounds/t

In [21]:
temp_test = pd.DataFrame(temp_test)
type(temp_test)

pandas.core.frame.DataFrame

In [22]:
temp_test.columns = ['mix']

In [23]:
temp_test.keys()

Index(['mix'], dtype='object')

In [24]:
temp_test[['ID','feature']] = temp_test['mix'].apply(pd.Series)

In [25]:
temp_test.drop('mix',axis=1,inplace=True)

In [26]:
print("\n---------------------train data---------------------")
print(type(temp))
print(temp.head())

print("\n---------------------val data---------------------")
print(type(temp_val))
print(temp_val.head())

print("\n---------------------test data---------------------")
print(type(temp_test))
print(temp_test.head())


print('---------------------Checking for NONE values---------------------')
# checking for NONE values
#print(temp[temp.Class.isnull()])

# removing NONE values from temp
temp = temp.dropna()
temp_val = temp_val.dropna()
temp_test = temp_test.dropna()
#temp = temp[temp.Class.notnull()]
#temp_test = temp_test[temp_test.notnull()]
#print(temp.ID[temp.label.isnull()])

#print("\n---------------------val data---------------------")
#print(type(temp_val))
#print(temp_val.head())


---------------------train data---------------------
<class 'pandas.core.series.Series'>
0    [0, [-82.12359, 139.50595, -42.430847, 24.8278...
2    [2, [-123.393654, 15.181944, -50.093338, 7.141...
4    [4, [-237.9265, 135.90247, 39.26844, 21.24023,...
5    [6, [-250.35838, 128.01352, -47.575783, 25.628...
7    [11, [-55.40057, 22.76732, -46.079865, 8.98009...
dtype: object

---------------------val data---------------------
<class 'pandas.core.series.Series'>
15    [24, [-199.52899, 183.66898, -7.14285, 10.1079...
16    [26, [-363.90073, 176.40643, 42.35497, 44.8453...
17    [27, [-315.29913, 184.21286, -6.5808682, -0.34...
19    [33, [-145.92226, 122.60997, 3.84362, 22.73473...
32    [49, [55.029873, 0.89304477, -46.641396, -6.57...
dtype: object

---------------------test data---------------------
<class 'pandas.core.frame.DataFrame'>
   ID                                            feature
0   5  [-76.2993, 20.437246, -17.56602, 26.38778, -22...
1   7  [-375.73608, 149.91734, -21

In [27]:
temp.Class.unique()

array(['siren', 'drilling', 'dog_bark', 'children_playing',
       'engine_idling', 'gun_shot', 'air_conditioner', 'jackhammer',
       'street_music', 'car_horn'], dtype=object)

In [28]:
temp.Class.nunique()

10

In [29]:
# Label Encoding the audio data
lb = LabelEncoder()

# converting pd.series into np.array for faster processing
X = np.array(temp.feature.tolist())
y = np.array(temp.Class.tolist())


y = to_categorical(lb.fit_transform(y))

In [30]:
# converting pd.series into np.array for faster processing
X_test = np.array(temp_val.feature.tolist())
y_test = np.array(temp_val.Class.tolist())

y_test = to_categorical(lb.transform(y_test))

In [31]:
#x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.3)

x_train=X
y_train=y

x_test=X_test
y_test=y_test


## Building a deep learning model

In [32]:
num_labels = y.shape[1]
filter_size = 2

def categorical_classifier():
    model = Sequential()

    # input and first hidden layer
    model.add(Dense(input_shape=(40,), units=256, activation='relu', kernel_initializer='uniform'))
    model.add(Dropout(0.5))

    # second hidden layer
    model.add(Dense(units=256,activation='relu',kernel_initializer='uniform'))
    model.add(Dropout(0.5))

    # output layer
    model.add(Dense(units=num_labels, activation='softmax'))

    # compiling our model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

    # training the data
    #model.fit(X,y, batch_size=32, epochs=500, validation_split=0.3)
    return model

In [33]:
model2=categorical_classifier()

In [34]:
# training the data
model2.fit(x_train,y_train, batch_size=32, epochs=200, validation_data=(x_test, y_test))


Train on 3209 samples, validate on 1086 samples
Epoch 1/200
3209/3209 [==============================] - 1s 248us/step - loss: 2.7068 - accuracy: 0.2487 - val_loss: 1.7220 - val_accuracy: 0.4540
Epoch 2/200
3209/3209 [==============================] - 0s 137us/step - loss: 1.7020 - accuracy: 0.4095 - val_loss: 1.3862 - val_accuracy: 0.5378
Epoch 3/200
3209/3209 [==============================] - 0s 136us/step - loss: 1.5105 - accuracy: 0.4771 - val_loss: 1.2789 - val_accuracy: 0.5783
Epoch 4/200
3209/3209 [==============================] - 0s 136us/step - loss: 1.3820 - accuracy: 0.5242 - val_loss: 1.1460 - val_accuracy: 0.6436
Epoch 5/200
3209/3209 [==============================] - 0s 136us/step - loss: 1.3077 - accuracy: 0.5510 - val_loss: 1.0555 - val_accuracy: 0.6611
Epoch 6/200
3209/3209 [==============================] - 0s 139us/step - loss: 1.2217 - accuracy: 0.5980 - val_loss: 0.9857 - val_accuracy: 0.6915
Epoch 7/200
3209/3209 [==============================] - 0s 138us/ste

3209/3209 [==============================] - 1s 163us/step - loss: 0.5623 - accuracy: 0.8015 - val_loss: 0.4864 - val_accuracy: 0.8527
Epoch 55/200
3209/3209 [==============================] - 0s 136us/step - loss: 0.5330 - accuracy: 0.8183 - val_loss: 0.4700 - val_accuracy: 0.8656
Epoch 56/200
3209/3209 [==============================] - 0s 138us/step - loss: 0.5416 - accuracy: 0.8133 - val_loss: 0.4692 - val_accuracy: 0.8582
Epoch 57/200
3209/3209 [==============================] - 0s 136us/step - loss: 0.5042 - accuracy: 0.8193 - val_loss: 0.4462 - val_accuracy: 0.8711
Epoch 58/200
3209/3209 [==============================] - 0s 137us/step - loss: 0.5261 - accuracy: 0.8227 - val_loss: 0.4409 - val_accuracy: 0.8748
Epoch 59/200
3209/3209 [==============================] - 0s 137us/step - loss: 0.5288 - accuracy: 0.8224 - val_loss: 0.4135 - val_accuracy: 0.8738
Epoch 60/200
3209/3209 [==============================] - 0s 144us/step - loss: 0.4982 - accuracy: 0.8246 - val_loss: 0.4214 

3209/3209 [==============================] - 0s 139us/step - loss: 0.3910 - accuracy: 0.8644 - val_loss: 0.3769 - val_accuracy: 0.8941
Epoch 110/200
3209/3209 [==============================] - 0s 139us/step - loss: 0.4388 - accuracy: 0.8486 - val_loss: 0.3766 - val_accuracy: 0.8785
Epoch 111/200
3209/3209 [==============================] - 0s 137us/step - loss: 0.3959 - accuracy: 0.8620 - val_loss: 0.3961 - val_accuracy: 0.8904
Epoch 112/200
3209/3209 [==============================] - 0s 137us/step - loss: 0.4400 - accuracy: 0.8535 - val_loss: 0.3696 - val_accuracy: 0.8867
Epoch 113/200
3209/3209 [==============================] - 0s 138us/step - loss: 0.4426 - accuracy: 0.8573 - val_loss: 0.3782 - val_accuracy: 0.8932
Epoch 114/200
3209/3209 [==============================] - 0s 136us/step - loss: 0.4206 - accuracy: 0.8554 - val_loss: 0.3601 - val_accuracy: 0.8904
Epoch 115/200
3209/3209 [==============================] - 0s 138us/step - loss: 0.4483 - accuracy: 0.8573 - val_loss: 0

3209/3209 [==============================] - 0s 141us/step - loss: 0.3859 - accuracy: 0.8719 - val_loss: 0.3591 - val_accuracy: 0.8996
Epoch 165/200
3209/3209 [==============================] - 0s 137us/step - loss: 0.3821 - accuracy: 0.8629 - val_loss: 0.3452 - val_accuracy: 0.8923
Epoch 166/200
3209/3209 [==============================] - 0s 138us/step - loss: 0.3700 - accuracy: 0.8707 - val_loss: 0.3513 - val_accuracy: 0.8785
Epoch 167/200
3209/3209 [==============================] - 0s 139us/step - loss: 0.3534 - accuracy: 0.8766 - val_loss: 0.3631 - val_accuracy: 0.8886
Epoch 168/200
3209/3209 [==============================] - 0s 140us/step - loss: 0.3886 - accuracy: 0.8704 - val_loss: 0.3722 - val_accuracy: 0.8867
Epoch 169/200
3209/3209 [==============================] - 0s 142us/step - loss: 0.3907 - accuracy: 0.8710 - val_loss: 0.3595 - val_accuracy: 0.8886
Epoch 170/200
3209/3209 [==============================] - 0s 137us/step - loss: 0.3958 - accuracy: 0.8722 - val_loss: 0

In [35]:
# evaluate the model
scores = model2.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))

accuracy: 90.24%


In [36]:
#Predictions

y_prob = model2.predict(x_test) 
y_classes = y_prob.argmax(axis=-1)

In [37]:
len(y_classes)

1086

In [38]:
col1 = np.array(temp_val.ID)
col2 = np.array(y_classes)

In [39]:
df = pd.DataFrame({"ID" : col1, "ClassPred" : col2})
df.to_csv("predictions/c-any2modelagreed-iter3.csv", index=False)